$This\ script\ is\ used\ for\ creating\ the\ primitive\ alloy\ structure.$

$Its\ function\ can\ be\ easily\ known.$

$However,\ if\ you\ want\ to\ use\ it\ to\ creat\ a\ 3D\ sphere\ figure\ of\ the\ alloy.$

$There\ are\ some\ things\ you\ need\ to\ first\ take\ care\ of.$

$Because\ the\ mayavi\ is\ used,\ so\ you\ need\ to\ the\ Python\ version\ is\ lower\ than\ 3.8\ and\ VTK's\ version\ is\ 8.1.2$

$It's\ recommended\ to\ follow\ the\ command\ as\ below:$
```
conda create -n myenv python=3.7
conda activate myenv  
pip install PyQt5 
pip3 install vtk==8.1.2
pip install mayavi
```

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from random import choice
from mpl_toolkits.mplot3d import Axes3D
from collections import Counter
from mayavi import mlab

In [2]:
supercell = [5,5,5]
atoms_dc = {
    "Nb": 1.46,             ##  atom radius (angstrom) 
    "V" : 1.34,             ##  atom radius (angstrom)
    "Ta": 1.46,             ##  atom radius (angstrom)
    "Mo": 1.39,             ##  atom radius (angstrom)
    "W" : 1.39,             ##  atom radius (angstrom)
}
atomcolor_dc = {
    "Nb": "slateblue",
    "V" : "red",
    "Ta": "teal",
    "Mo": "fuchsia",
    "W" : "lawngreen"
}
atommayavi = {
    "Nb": (0,0.5,0.2),
    "V" : (0,0.1,0.9),
    "Ta": (0,0.3,0.7),
    "Mo": (0.4,0.6,1),
    "W" : (0.8,0.5,0.3)
}

In [3]:
def _random_mixing(inp1, inp2 , inp3, plotting_style, *args):
    """
    Constructing the random multicomponent alloy in roughly equal proportions
    inp1 = corresponding basis
    inp2 = the unit cell enlargment
    inp3 = cystal structure type
    plotting _style = "dot" or "sphere"
    *args = "Na","V",...
    """
    # Produce the Alloy Crystal
    x = np.arange(0,inp2[0],1)
    y = np.arange(0,inp2[1],1)
    z = np.arange(0,inp2[2],1)
    X,Y,Z = np.meshgrid(x,y,z)
    diceinp = list(range(len(args)))
    out = []
    translation = np.column_stack((X.reshape(-1,1),Y.reshape(-1,1),Z.reshape(-1,1)))
    for i in tqdm(range(translation.shape[0])):
        trans = translation[i,:]
        temp = list(map(lambda x:np.append(x+trans,[[choice(diceinp)]],axis=-1),inp1))
        out.extend(temp)
    outarray = np.concatenate(out,axis=0) 
    outarray = outarray[np.lexsort((outarray[:,-1],))]
    np.savetxt("grid.csv",outarray,delimiter = ",", fmt = "%.6f")
    
    # Counting each atom's number
    numberdc = dict(Counter(outarray[:,-1]))
    temp = {}
    for key in sorted(numberdc.keys()):
        temp[key] = numberdc[key]
    numberdc = temp
    print(numberdc)

    # Plotting The Alloy 
    r = lambda x : atoms_dc[args[int(x)]]        
    color = lambda x : atomcolor_dc[args[int(x)]]
    vecr = np.vectorize(r)
    veccolor = np.vectorize(color)
    if plotting_style == "dot":
        fig = plt.figure()
        ax = Axes3D(fig,)
        print("This is based on Matplotlib 3D.")
        ax.scatter(outarray[:,0],outarray[:,1],outarray[:,2],\
                   s = vecr(outarray[:,3])*50, c = veccolor(outarray[:,3]) , marker="o" )
    elif plotting_style == "sphere":
        print("This is based on Mayavi.\nSo please ensure that import line is uncommented.")
        print("Please Use the Python below 3.8 and install the right version of VTK using pip install vtk==8.1.2")
        mlab.points3d(outarray[:,0],outarray[:,1],outarray[:,2],\
                      outarray[:,3]+1, colormap="cool", scale_factor=.25 )
        mlab.savefig("1.png", size=(400,300), )
   # np.where((outarray[:,3]==0),10*outarray[:,3],outarray[:,3]) 
    # Calculating the average lattice parameter
    numerator = 0; denomiter = 0
    if inp3 != "hcp":                  # here we make an approximation and maybe later it'll be amended
        for key,value in numberdc.items():
            numerator += atoms_dc[args[int(key)]]*value
            denomiter += value 
        average_lattice_parameter = 2*numerator/denomiter

    # Writting POSCAR file for ezvasp    
    with open("POSCAR_{}_{}".format("".join(args),inp3),"w") as f:
        f.write("".join(args)+"\n"+"  1\n")
        f.write("    {:12.12f}    {:12.12f}    {:12.12f}    \n".format(supercell[0]*average_lattice_parameter, 0, 0))
        f.write("    {:12.12f}    {:12.12f}    {:12.12f}    \n".format(0, supercell[1]*average_lattice_parameter, 0))
        f.write("    {:12.12f}    {:12.12f}    {:12.12f}    \n".format(0, 0, supercell[2]*average_lattice_parameter))
        f.write("   {}\n".format("  ".join(list(map(lambda x:args[int(x)],numberdc.keys())))))
        f.write("   {}\n".format("  ".join(list(map(str,numberdc.values())))))
        f.write("Direct\n")
        for i in range(outarray.shape[0]):
            f.write("    {:12.12f}    {:12.12f}    {:12.12f}    \n".format(outarray[i,0]/supercell[0],outarray[i,1]/supercell[1],outarray[i,2]/supercell[2],) )
    
    # Writting rndstr.in for mcsqs
    with open("rndstr.in_{}_{}".format("".join(args),inp3),"w") as f:
        f.write("{:12.12f}    {:12.12f}    {:12.12f}    {:12d}    {:12d}    {:12d}\n".format(average_lattice_parameter, average_lattice_parameter, average_lattice_parameter, 90, 90, 90))
        f.write("1    0    0\n0    1    0\n0    0    1\n")
        for i in range(outarray.shape[0]):
            f.write("{:12.5f}    {:12.5f}    {:12.5f}    {}\n".format( outarray[i,0],outarray[i,1],outarray[i,2],args[int(outarray[i,3])] ))

##### $Simple\ Cubic\ Structure$

In [4]:
scbasis = [np.array([[0,0,0]])]

##### $BCC\ Structure$

In [5]:
bccbasis = [np.array([[0,0,0]]), np.array([[0.5,0.5,0.5]])]

##### $FCC\ Structure$

In [6]:
fccbasis = [np.array([[0,0,0]]), np.array([[0.5,0,0.5]]), np.array([[0,0.5,0.5]])]

##### $HCP\ Structure$ 

In [7]:
_random_mixing(bccbasis, supercell, "bcc" ,"sphere","Nb","V","Ta")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 24149.61it/s]


{0.0: 88, 1.0: 81, 2.0: 81}
This is based on Mayavi.
So please ensure that import line is uncommented.
Please Use the Python below 3.8 and install the right version of VTK using pip install vtk==8.1.2
